# Interest Rate Swap Pricing

An **Interest Rate Swap** is a financial derivative instrument in which two parties agree to exchange interest rate cash flows based on a notional amount from a fixed rate to a floating rate or from one floating rate to another floating rate.

Here we will consider an example of a plain vanilla USD swap with **10 million notional** and **10 year maturity**. Let the **fixed leg pay 2.5% coupon semiannually**, and the **floating leg pay Libor 3m quarterly**

Here we construct the yield curve objects. For simplicity, we will use **flat curves for discounting** and **Libor 3M**. This will help us focus on the Swap construction part

In [12]:
import QuantLib as ql

def print_curve(xlist, ylist, precision=3):
    """
    Method to print curve in a nice format
    """
    print("----------------------")
    print("Maturities\tCurve")
    print("----------------------")
    for x,y in zip(xlist, ylist):
        print(x,"\t\t", round(y, precision))
    print("----------------------")
    
    

The deposit rates and fixed rate bond rates are provided below. This example is based on Exhibit 5-5 given in Frank Fabozzi's Bond Markets, Analysis and Strategies, Sixth Edition.



In [13]:
# some constants and conventions
# here we just assume for the sake of example
# that some of the constants are the same for
# depo rates and bond rates

calc_date = ql.Date(15, 1, 2015)
ql.Settings.instance().evaluationDate = calc_date

In [14]:
# construct discount curve and libor curve

risk_free_rate = 0.01
libor_rate = 0.02
day_count = ql.Actual365Fixed()

discount_curve = ql.YieldTermStructureHandle(ql.FlatForward(calc_date, risk_free_rate, day_count))
libor_curve = ql.YieldTermStructureHandle(ql.FlatForward(calc_date, libor_rate, day_count))
#libor3M_index = ql.Euribor3M(libor_curve)  
libor3M_index = ql.USDLibor(ql.Period(3, ql.Months), libor_curve)

To **construct the Swap instrument**, we have to specify the **fixed rate leg** and **floating rate leg**. We construct the **fixed rate** and **floating rate leg schedules** below.

In [15]:
calendar = ql.UnitedStates(ql.UnitedStates.NYSE)
settle_date = calendar.advance(calc_date, 5, ql.Days)
maturity_date = calendar.advance(settle_date, 10, ql.Years)

fixed_leg_tenor = ql.Period(6, ql.Months)
fixed_schedule = ql.Schedule(settle_date, maturity_date, 
                             fixed_leg_tenor, calendar,
                             ql.ModifiedFollowing, ql.ModifiedFollowing,
                             ql.DateGeneration.Forward, False)

float_leg_tenor = ql.Period(3, ql.Months)
float_schedule = ql.Schedule (settle_date, maturity_date, 
                              float_leg_tenor, calendar,
                              ql.ModifiedFollowing, ql.ModifiedFollowing,
                              ql.DateGeneration.Forward, False)

Below, we construct a **VanillaSwap object** by including the fixed and float leg schedules created above.

In [16]:
notional = 10000000
fixed_rate = 0.025
fixed_leg_daycount = ql.Actual360()
float_spread = 0.004
float_leg_daycount = ql.Actual360()

ir_swap = ql.VanillaSwap(ql.VanillaSwap.Payer, notional, fixed_schedule, 
               fixed_rate, fixed_leg_daycount, float_schedule,
               libor3M_index, float_spread, float_leg_daycount )

We **evaluate the swap using a discounting engine.**

In [17]:
swap_engine = ql.DiscountingSwapEngine(discount_curve)
ir_swap.setPricingEngine(swap_engine)

## Fixed Leg Cashflows

In [18]:
for i, cf in enumerate(ir_swap.leg(0)):
    print("%2d %-18s %10.2f"%(i+1, cf.date(), cf.amount()))

 1 July 23rd, 2015     125694.44
 2 January 25th, 2016  129166.67
 3 July 25th, 2016     126388.89
 4 January 23rd, 2017  126388.89
 5 July 24th, 2017     126388.89
 6 January 23rd, 2018  127083.33
 7 July 23rd, 2018     125694.44
 8 January 23rd, 2019  127777.78
 9 July 23rd, 2019     125694.44
10 January 23rd, 2020  127777.78
11 July 23rd, 2020     126388.89
12 January 25th, 2021  129166.67
13 July 23rd, 2021     124305.56
14 January 24th, 2022  128472.22
15 July 25th, 2022     126388.89
16 January 23rd, 2023  126388.89
17 July 24th, 2023     126388.89
18 January 23rd, 2024  127083.33
19 July 23rd, 2024     126388.89
20 January 23rd, 2025  127777.78


## Float Leg Cashflow

In [19]:
for i, cf in enumerate(ir_swap.leg(1)):
    print("%2d %-18s %10.2f"%(i+1, cf.date(), cf.amount()))

 1 April 23rd, 2015     59436.87
 2 July 23rd, 2015      60098.65
 3 October 23rd, 2015   60760.46
 4 January 25th, 2016   62084.17
 5 April 25th, 2016     60098.65
 6 July 25th, 2016      60098.65
 7 October 24th, 2016   60098.65
 8 January 23rd, 2017   60098.65
 9 April 24th, 2017     60098.65
10 July 24th, 2017      60098.65
11 October 23rd, 2017   60098.65
12 January 23rd, 2018   60760.46
13 April 23rd, 2018     59436.87
14 July 23rd, 2018      60098.65
15 October 23rd, 2018   60760.46
16 January 23rd, 2019   60760.46
17 April 23rd, 2019     59436.87
18 July 23rd, 2019      60098.65
19 October 23rd, 2019   60760.46
20 January 23rd, 2020   60760.46
21 April 23rd, 2020     60098.65
22 July 23rd, 2020      60098.65
23 October 23rd, 2020   60760.46
24 January 25th, 2021   62084.17
25 April 23rd, 2021     58113.40
26 July 23rd, 2021      60098.65
27 October 25th, 2021   62084.17
28 January 24th, 2022   60098.65
29 April 25th, 2022     60098.65
30 July 25th, 2022      60098.65
31 October

## NPV 

In [20]:
print("%-20s: %20.3f" % ("Net present Value", ir_swap.NPV()))
print("%-20s: %20.3f" % ("Fair Spread", ir_swap.fairSpread()))
print("%-20s: %20.3f" % ("Fair Rate", ir_swap.fairRate()))
print("%-20s: %20.3f" % ("Fixed Leg BPS", ir_swap.fixedLegBPS()))
print("%-20s: %20.3f" % ("Float Leg BPS", ir_swap.floatingLegBPS()))

Net present Value   :          -115050.676
Fair Spread         :                0.005
Fair Rate           :                0.024
Fixed Leg BPS       :            -9629.716
Float Leg BPS       :             9641.779
